This notebook uses a pretrained model (inception v3) to classify plants as invasive or not.
---------------------------
I ended up getting to 92% accuracy after 10 epochs. I only trained the pretrained weights at a low learning rates every fifth epoch.



The reason for the file structure is I ran this on a kaggle kernal initially.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import pandas as pd
import cv2
import math
from glob import glob
import os
from sklearn import metrics
import glob2


## resize train data and test data ##

In [2]:
img_path = "../input/train/"

y = []
file_paths = []
for i in range(len(master)-750):
    file_paths.append( img_path + str(master.loc[i][0]) +'.jpg' )
    y.append(master.loc[i][1])
y = np.array(y)

In [3]:
from pathlib import Path
PATH = Path(".")

def get_optimizer(model, lr = 0.01, wd = 0.0):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch.optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))
def get_triangular_lr(lr_low, lr_high, stepesize):
    iterations = 2*stepesize
    iter1 = int(0.35*iterations)
    iter2 = int(0.85*iter1)
    iter3 = iterations - iter1 - iter2
    delta1 = (lr_high - lr_low)/iter1
    delta2 = (lr_high - lr_low)/(iter1 -1)
    lrs1 = [lr_low + i*delta1 for i in range(iter1)]
    lrs2 = [lr_high - i*(delta1) for i in range(0, iter2)]
    delta2 = (lrs2[-1] - lr_low)/(iter3)
    lrs3 = [lrs2[-1] - i*(delta2) for i in range(1, iter3+1)]
    return lrs1+lrs2+lrs3
def train_triangular_policy(model, train_dl, valid_dl, lr_low=1e-5, lr_high=0.01):
    idx = 0
    epochs = 1
    stepesize = 2*len(train_dl)
    lrs = get_triangular_lr(lr_low, lr_high, stepesize)

    model.train()
    total = 0
    sum_loss = 0
    for i, (mod_input) in enumerate(train_dl):
        x,y = mod_input['x'],mod_input['y']

        optim = get_optimizer(model, lr = lrs[idx], wd =0)
        batch = y.shape[0]
        x = x.cuda().float()
        y = y.cuda().float().unsqueeze(1)
        out = model(x)[0]
        loss =  F.binary_cross_entropy_with_logits(out, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        idx += 1
        total += batch
        sum_loss += batch*(loss.item())
    print("train loss", sum_loss/total)
    train_loss=sum_loss/total

        

def training_loop(model, train_dl, valid_dl, steps=3, lr_low=1e-5, lr_high=0.01):

    start = datetime.now() 
    train_loss = train_triangular_policy(model, train_dl, valid_dl, lr_low, lr_high)
    end = datetime.now()
    t = 'Time elapsed {}'.format(end - start)
    return train_loss

In [4]:
#image reseize & centering & crop 

import math
def center_crop(im, min_sz=299):
    """ Returns a center crop of an image"""
    """ Returns a random crop"""
    r,c,*_ = im.shape
    start_c = 0
    print(im.shape)
    if im.shape[2]< im.shape[1]:

        return crop(im, 0, start_c, min_sz, min_sz)
    else:

        
        return crop(im,start_c, 0, min_sz, min_sz)

def crop(im, r, c, target_r, target_c): return im[:,r:r+target_r, c:c+target_c]

def random_crop(x,target_size = 2):
    """ Returns a random crop"""
    r,c,*_ = x.shape
    rand_c = np.random.uniform(0, 1)*32
    start_c = np.floor(rand_c).astype(int)
    
    if x.shape[2]< x.shape[1]:
        return crop(x, 0, start_c, target_size, target_size)
    else:
        return crop(x,start_c, 0, target_size, target_size)

def rotate_cv(im,  mode=cv2.BORDER_REFLECT, interpolation=cv2.INTER_AREA):
    """ Rotates an image by deg degrees"""
    r,c,*_ = im.shape
    deg = np.random.rand() * 180
    M = cv2.getRotationMatrix2D((c/2,r/2),deg,1)

    return cv2.warpAffine(im,M,(c,r), borderMode=mode, flags=cv2.WARP_FILL_OUTLIERS+interpolation)

def random_flip(x):
    p = np.random.rand()
    if p>0.5:
        return np.flip(x,1).copy()
    else:
        return x


save numpy array.

Usually I separate code, data format and CNN.

## split train data and validation data  ##

In [5]:
import torch
from torch.utils.data import DataLoader,Dataset
from torch import nn
import torch.nn.functional as F

In [6]:
class FlowerDataset(Dataset):
    def __init__(self,img_paths,y,transform=False):
        self.X=img_paths
        self.y=y
        if transform: self.transform = lambda x: random_flip(rotate_cv(x))
        else: self.transform = lambda x: x

    def __getitem__(self,idx):
        img = cv2.imread(self.X[idx])
        img = img.astype('float64')

        img[:,:,0] -= 103.939
        img[:,:,1] -= 116.779
        img[:,:,2] -= 123.68
        img /= 255.
        #resize 
        img = cv2.resize(img,(299,299)).T 
        return {'x':self.transform(img),'y':self.y[idx]}
    
    def __len__(self):
        return len(self.y)
class TestFlowerDataset(Dataset):
    def __init__(self,img_paths,transform=False):
        self.X=img_paths
        self.y=y
        if transform: self.transform = lambda x: random_flip(rotate_cv(x))
        else: self.transform = lambda x: x

    def __getitem__(self,idx):
        img = cv2.imread(self.X[idx])
        img = img.astype('float64')

        img[:,:,0] -= 103.939
        img[:,:,1] -= 116.779
        img[:,:,2] -= 123.68
        img /= 255.
        #resize 
        img = cv2.resize(img,(299,299)).T 
        return {'x':self.transform(img),'y':self.y}
    
    def __len__(self):
        return len(self.y)

Validation Accuracy

# Using a Pretrained Model

In [1]:


def val_metrics(model, valid_dl):
    model.eval()
    total = 0
    sum_loss = 0
    correct = 0 
    for input in valid_dl:
        model.eval()

        x = input['x'].cuda()
        y = input['y'].long().cuda()
        batch = y.shape[0]
        x = x.cuda().float()
        y = y.cuda().unsqueeze(1)
        out = model(x)
        pred = (out > 0.0).long()
        correct += pred.eq(y.data).sum().item()
        y = y.float()
        loss = F.binary_cross_entropy_with_logits(out, y)
        sum_loss += batch*(loss.item())
        total += batch
    print("val loss and accuracy", sum_loss/total, correct/total)
    return correct/total


def eval(model, valid_dl):
    total = 0
    sum_loss = 0
    probs = []
    ys= []
    for input in valid_dl:
        model.eval()

        x = input['x'].cuda()
        y = input['y'].float().cuda()
        batch = y.shape[0]
        
        y_hat = model(x.float()).flatten() 
    
        
        probs.append(int(y_hat[0].cpu().detach().numpy()>0.5))

        ys.append(int(y.cpu().detach().numpy()[0]))

        loss = F.binary_cross_entropy_with_logits(y_hat,y).cuda()
        sum_loss += batch * loss.item()
        total+=batch

    probs = np.vstack(probs)
    ys = np.vstack(ys)
    
    acc = metrics.accuracy_score(ys, probs)
    torch.cuda.empty_cache()
    return sum_loss/total, acc    

In [8]:
from torchvision import models

In [2]:
class Reshape(nn.Module):
    def __init__(self, *args):
        super(Reshape, self).__init__()
        self.shape = args

    def forward(self, x):
        return x.view(self.shape)
        
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        inception = models.inception_v3(pretrained='imagenet',transform_input=True) 
        # freezing parameters
        for i, param in inception.named_parameters():
            param.requires_grad = False
            

        in_filters = inception.fc.in_features
        
        # convolutional layers of resnet34

        
        last_layer = nn.Sequential(*[
            nn.ReLU(),
            nn.BatchNorm1d(in_filters),    
            nn.Linear(in_filters, in_filters),
            nn.ReLU(),
            nn.BatchNorm1d(in_filters),
            nn.Linear(in_filters, 1)])
        inception.fc = last_layer
        self.inception = inception
    
    def forward(self, x):
        x = self.inception(x)
        print(type(x))
        return torch.Tensor(x)
    def freeze(self):
        for i, param in self.inception.named_parameters():
            param.requires_grad = False
    def unfreeze(self):
        for i, param in self.inception.named_parameters():
            param.requires_grad = True
    

NameError: name 'nn' is not defined

In [ ]:
model = Net().cuda()
epochs = 1000
learning_rate = 0.001
weight_decay=0.001
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor=0.25, patience=1,cooldown=1, min_lr=1e-7,verbose=True)
train_input,val_input,train_output,val_output = train_test_split(file_paths,y)

In [3]:
train_dataset= FlowerDataset(train_input,train_output,transform=True)
val_dataset= FlowerDataset(val_input,val_output,transform=False)


NameError: name 'FlowerDataset' is not defined

In [44]:
train_dataloader = DataLoader(train_dataset,batch_size=12,shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size=12,shuffle=False)

In [45]:
from datetime import datetime

torch.nn.

In [47]:
train_losses=[]
val_losses=[]
last_acc=0
val_accs=[]
lr_low=0.0005
lr_high=0.002
for epoch in range(15):
    avg_val_acc = 0
    avg_train_loss = 0 
    avg_val_loss = 0
       
    total_loss=0
    total=0
    if epoch%5 == 0:
        model.unfreeze()
        train_loss = training_loop(model, train_dataloader, val_dataloader, steps=1, lr_low=1e-5,lr_high=0.0005 )
        model.freeze()
    else:
        model.unfreeze()
        train_loss = training_loop(model, train_dataloader, val_dataloader, steps=1, lr_low=lr_low,lr_high=lr_high)
        model.freeze()
    val_acc = val_metrics(model, val_dataloader)
    val_losses.append(val_acc)

#     if val_acc < last_acc:
#         lr_low*=0.8
#         lr_high*=0.8
#         print('Shrinking Learning Rate to:',str(lr_low),'to,',str(lr_high))
    last_acc = val_acc



train loss 0.5278502881835779
val loss and accuracy 0.4389747302084006 0.9121447028423773
train loss 0.6760029819036395
val loss and accuracy 2.1288044434185176 0.648578811369509
train loss 0.4882872643557237
val loss and accuracy 1.8981971070747967 0.8191214470284238
train loss 0.40129411205108917
val loss and accuracy 0.4540959900432779 0.7984496124031008
train loss 0.39882178882551933
val loss and accuracy 19.723487713540248 0.4160206718346253
train loss 0.32844588901712485
val loss and accuracy 0.31558458160522374 0.8682170542635659
train loss 0.40021285738969714
val loss and accuracy 0.42350348576094754 0.8552971576227391
train loss 0.39202842791926673
val loss and accuracy 0.700088641902273 0.6330749354005168
train loss 0.4348357002506602
val loss and accuracy 0.3407989811989688 0.8578811369509044
train loss 0.3771199317639356
val loss and accuracy 66.05415870607361 0.3695090439276486
train loss 0.3220565038023835
val loss and accuracy 0.2690491846134496 0.8811369509043928
train 

In [ ]:
def load_img(paths):
    imf_d = {}
    for f in paths:
        imf_d[f] = f
    fdata = [imf_d[f] for f in paths]
    return fdata

In [ ]:
torch.save(model,f='new_mod')

In [ ]:
test_jpg = glob('../input/test/*.jpg')
test = pd.DataFrame([[p.split('/')[3].replace('.jpg',''),p] for p in test_jpg])
test.columns = ['name','path']
xtest = load_img(test['path']); print('test...')
pd.DataFrame.from_dict(xtest).to_csv('xtest1.csv', index=False)
xtest = pd.read_csv('xtest1.csv')

In [ ]:
test_p = '../input/test/'


file_paths = []
for i in range(len(xtest)):
    file_paths.append( str(xtest.loc[i][0]) )
    

In [ ]:
predictions = [0 for x in range(len(file_paths))]
name = lambda x:int( x.split('/')[-1].split('.')[0])
for x in file_paths:
    print(name(x))
    model.eval()
    img = cv2.imread(x)
    img = img.astype('float64')

    img[:,:,0] -= 103.939
    img[:,:,1] -= 116.779
    img[:,:,2] -= 123.68
    img /= 255.
    #resize 
    img = np.rollaxis(cv2.resize(img,(299,299)),2)
    img=torch.Tensor(random_flip(rotate_cv(img))).unsqueeze(0).cuda()
    predictions[name(x)] = model(img)

In [ ]:
[int(pred) for pred in predictions]

In [ ]:
filename = 'new.csv'
with open(filename,'w+') as f:
    f.write("name,invasive\n")
    for i, prediction in enumerate([int(pred) for pred in predictions]):
        line = str(i)+','+str(prediction)+'\n'
        f.write(line)

## My personal Model

In [ ]:
model = nn.Sequential(*[nn.Conv2d(3,10,kernel_size=2,stride=2),\
                       nn.ReLU(True),\
                       nn.BatchNorm2d(10),#128
                        nn.Conv2d(10,9,kernel_size=2,stride=2),\
                       nn.ReLU(True),\
                       nn.BatchNorm2d(9),#64
                        nn.Conv2d(9,8,kernel_size=2,stride=2),\
                       nn.ReLU(True),\
                       nn.BatchNorm2d(8),   #32
                        nn.Conv2d(8,7,kernel_size=2,stride=2),\
                       nn.ReLU(True),\
                       nn.BatchNorm2d(7),  #16
                        nn.Conv2d(7,6,kernel_size=2,stride=2),\
                       nn.ReLU(True),\
                       nn.BatchNorm2d(6),  #8
                        nn.Conv2d(6,5,kernel_size=2,stride=2),\
                       nn.ReLU(True),\
                       nn.BatchNorm2d(5),  #4
                        nn.Conv2d(5,1,kernel_size=2,stride=2)
                       ]).cuda()

In [ ]:
learning_rate = 0.0005
weight_decay=0.001
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=weight_decay)

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a random sample dataframe
df = pd.read_csv('new.csv')

# create a link to download the dataframe
create_download_link(df)